In [0]:

%sql
CREATE TABLE IF NOT EXISTS Pipeline_Run_Audit_Delta (
  SourceType STRING,
  FileName STRING,
  Status STRING,
  StartTime TIMESTAMP,
  EndTime TIMESTAMP
)
USING DELTA
LOCATION 'abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/audit_logs/Pipeline_Run_Audit_Delta';

In [0]:
from datetime import datetime
from pyspark.sql import Row
from pyspark.sql.functions import explode, col
# Step 1: Capture start time
start_time = datetime.now()
# Step 2: Initialize status
status = "Success"
error_message = ""
 
# Step 3: Wrap ingestion logic
try:
    # Retrieve file name from widget
    file_name = dbutils.widgets.get("p_file_name")
 
    # Extract base name without extension
    base_name = file_name.split('.')[0]
 
    # Create table name with prefix
    table_name = f"bronze_{base_name}"
 
    # Define Delta path
    delta_path = f"abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/Bronze/{base_name}_delta"
 
    # Read CSV and write to Delta
    df = spark.read.format("csv") \
        .option("multiline", True) \
        .option("header", True) \
        .option("inferSchema", True) \
        .load(f"abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/raw/{file_name}")
 
    df.write.mode("overwrite") \
        .option("mergeSchema", "true") \
        .format("delta") \
        .save(delta_path)
 
    # Register Delta table
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {table_name}
        USING DELTA
        LOCATION '{delta_path}'
    """)
 
    spark.sql(f"SELECT * FROM {table_name}").show()
 
except Exception as e:
    status = "Failed"
    error_message = str(e)
finally:
    # Step 5: Create audit row
    end_time = datetime.now()
    audit_row = [Row(
        SourceType="CSV",  
        FileName=file_name,
        Status=status,
        StartTime=start_time,
        EndTime=end_time
 
 
 
    )]
    audit_df = spark.createDataFrame(audit_row)
 
    # Step 6: Write audit log to Delta
    audit_log_path = "abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/audit_logs/Pipeline_Run_Audit_Delta"
    audit_df.write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .save(audit_log_path)
 
    # Step 7: Register audit table (only once per notebook)
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS Pipeline_Run_Audit_Delta
        USING DELTA
        LOCATION '{audit_log_path}'
    """)
    spark.sql("SELECT * FROM Pipeline_Run_Audit_Delta").show()



    

+----------+------+---------+---------+--------+-------+
|      Date|Region| Category|Total_kWh|Peak_kWh|Min_kWh|
+----------+------+---------+---------+--------+-------+
|2025-11-01|    UK|  Turbine|    12425|  520.62| 138.83|
|2025-11-01|    UK|    Panel|    11684|   564.1| 163.89|
|2025-11-01|    UK|  Battery|     8384|   466.6| 127.89|
|2025-11-01|    UK|Generator|    11987|  423.68| 156.91|
|2025-11-01|    EU|  Turbine|     9817|  522.42| 163.45|
|2025-11-01|    EU|    Panel|    13778|  641.45| 171.85|
|2025-11-01|    EU|  Battery|     7598|  454.95| 123.65|
|2025-11-01|    EU|Generator|    10916|   410.9| 145.77|
|2025-11-01|  APAC|  Turbine|    11978|  459.45| 151.85|
|2025-11-01|  APAC|    Panel|    12404|   523.1| 135.69|
|2025-11-01|  APAC|  Battery|    10566|  429.15| 130.44|
|2025-11-01|  APAC|Generator|     8054|  377.35| 129.69|
|2025-11-01|    NA|  Turbine|    14590|  471.75| 163.27|
|2025-11-01|    NA|    Panel|    13735|  517.38| 154.57|
|2025-11-01|    NA|  Battery|  